In [ ]:
import json
with open("outputs/buckets_07-24.json","r") as file:
  buckets = json.load(file)
bucket_map={action:bucket[0] for bucket in buckets for action in bucket}
inv_bucket_map={bucket[0]:bucket for bucket in buckets}
bucket_names = [bucket[0] for bucket in buckets]

In [ ]:
import pandas as pd
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
for h in history_df_by_term:
  h["bucket"]=h["action"].apply(lambda action:bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())

In [ ]:
import networkx as nx
g_dict={}
for bucket1 in bucket_names:
  g_dict[bucket1]={}
  for bucket2 in bucket_names:
    g_dict[bucket1][bucket2]=0
for bill_id in bill_ids:
  buckets = list(bills[bill_id]["bucket"])
  for i in range(len(buckets)):
    for j in range(i+1,len(buckets)):
      g_dict[buckets[i]][buckets[j]]+=1
g=nx.DiGraph()
for bucket1 in bucket_names:
  for bucket2 in bucket_names:
    if g_dict[bucket1][bucket2]>0:
      g.add_edge(bucket1,bucket2,weight=g_dict[bucket1][bucket2])
# del g_dict

In [ ]:
def usually_comes_before(bucket1,bucket2):
  return g_dict[bucket1][bucket2]-g_dict[bucket2][bucket1]


In [ ]:
def check_comparator(comparator_func, test_data):
    """
    Checks if a given comparator function appears to define a valid total order
    based on a provided set of test data.
    """
    # Check Reflexivity
    for x in test_data:
        if comparator_func(x, x) != 0:
            print(f"Reflexivity failed: cmp({x}, {x}) != 0")
            return False

    # Check Anti-symmetry and Transitivity (simplified check)
    for i in range(len(test_data)):
        for j in range(len(test_data)):
            a, b = test_data[i], test_data[j]
            cmp_ab = comparator_func(a, b)
            cmp_ba = comparator_func(b, a)
            a_display=a[:20]
            b_display=b[:20]
            c_display=c[:20]

            if cmp_ab < 0 and not (cmp_ba > 0):
                print(f"Anti-symmetry failed A: cmp({a_display}, {b_display}) < 0 but cmp({b_display}, {a_display}) not > 0")
                # return False
            if cmp_ab > 0 and not (cmp_ba < 0):
                print(f"Anti-symmetry failed B: cmp({a_display}, {b_display}) > 0 but cmp({b_display}, {a_display}) not < 0")
                # return False
            if cmp_ab == 0 and not (cmp_ba == 0):
                print(f"Anti-symmetry failed C: cmp({a_display}, {b_display}) == 0 but cmp({b_display}, {a_display}) != 0")
                # return False

            for k in range(len(test_data)):
                c = test_data[k]
                cmp_bc = comparator_func(b, c)
                cmp_ac = comparator_func(a, c)

                # Transitivity check (simplified)
                if cmp_ab < 0 and cmp_bc < 0 and not (cmp_ac <= 0):
                    a=a[:20]
                    b=b[:20]
                    c=c[:20]
                    display(f"Transitivity failed A: {a_display}<{b_display}, {b_display}<{c_display}, but not {a_display}<{c_display}")
                    # return False
                # if cmp_ab == 0 and cmp_bc == 0 and not (cmp_ac == 0):
                #     print(f"Transitivity failed B: {a}={b}, {b}={c}, but not {a}={c}")
                    # return False

    # return True
check_comparator(usually_comes_before,bucket_names)

In [ ]:
print(g.number_of_nodes())